In [5]:
# Raster Alignment Demonstrat+ion Script for AfricaPolis Data
# This script demonstrates how to align rasters to a reference grid using the geoworkflow package

import sys
from concurrent.futures import ProcessPoolExecutor, as_completed
from typing import List, Tuple, Optional
from pathlib import Path
import logging
# Import the necessary classes from geoworkflow
try:
    from geoworkflow.processors.spatial.clipper import ClippingProcessor
    from geoworkflow.processors.aoi.processor import AOIProcessor
    from geoworkflow.processors.integration.enrichment import StatisticalEnrichmentProcessor
    from geoworkflow.schemas.config_models import (
        AOIConfig, ClippingConfig, AlignmentConfig, StatisticalEnrichmentConfig, ResamplingMethod
    )
    from geoworkflow.processors.spatial.aligner import AlignmentProcessor, align_rasters
    from geoworkflow.core.base import ProcessingResult
    from geoworkflow.core.logging_setup import setup_logging
    
except ImportError as e:
    print(f"Error importing GeoWorkflow modules: {e}")
    print("Please ensure you're running from the project root with the correct environment activated.")
    sys.exit(1)

# Set up logging for better visibility of the process
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

## Defining and creating an AOI
AOI, area of interest, defines what area you want to analyze. The general input is a geospatial vector file (i.e., shp, geojson, gpkg) and the output is another geospatial vector file. 
Currently, our analyses have focused on agglomorations or countries. AOIProcessor can handle these or any other vectors. Users don't have to always create an AOI for every study. For example, if you are studying only Kenya, Tanzania, Ghana, and Togo, then you could theoretically just use the output file generated below in study after study, without every having to run the following block code again.

In [6]:

# start by configuring the AOI 
aoi_config = AOIConfig(
    input_file="../data/00_source/boundaries/africa_boundaries.geojson",
    country_name_column="NAME_0",  # country name column in geojson above
    countries=["Togo", "Ghana", "Tanzania", "Kenya"], # these country names must match names in NAME_0
    dissolve_boundaries=True,  # Combine all countries into single polygon
    buffer_km=50,  # Small buffer to capture env. data from boundaries
    output_file="../data/aoi/TogGhaTanKen50km.geojson" # target countries above, now dissolved and with a 50 km buffer
)

# Create the AOI
processor = AOIProcessor(aoi_config) 
result = processor.process() 

2025-09-18 22:41:56,717 - INFO - Starting AOIProcessor processing


Output()

2025-09-18 22:41:59,637 - INFO - Loading administrative boundaries


2025-09-18 22:42:02,520 - INFO - Filtering 4 countries
2025-09-18 22:42:02,522 - INFO - Dissolving country boundaries into single polygon


2025-09-18 22:42:03,208 - INFO - Applying 50.0 km buffer
2025-09-18 22:42:03,208 - WARNING - Using Africa Albers Equal Area Conic (ESRI:102022) for buffering operations. This projection is optimized for African datasets and may not be accurate for other continents. For non-African data, consider using a more appropriate projected coordinate system (e.g., UTM zones or continental equal-area projections).
2025-09-18 22:42:03,209 - INFO - Reprojecting from EPSG:4326 to ESRI:102022 for buffering
2025-09-18 22:42:03,222 - INFO - Applying 50.0km (50000.0m) buffer


2025-09-18 22:42:05,250 - INFO - Reprojecting back to EPSG:4326
2025-09-18 22:42:05,252 - INFO - Saving AOI to ../data/aoi/TogGhaTanKen50km.geojson
2025-09-18 22:42:05,261 - INFO - Created 1 records


2025-09-18 22:42:05,268 - INFO - AOI saved successfully completed: 4/6 items in 5.6s (0.7 items/sec)
2025-09-18 22:42:05,269 - INFO - Successfully completed AOIProcessor processing


## Clipping data to AOI


Clipping tif and vector files can be useful for reducing computational requirements, making further algorithms faster. The following functionality enables users to clip multiple files at once, just by pointing at a parent directory. Note that the function is recursive. Setting `input_directory = "../data/01_extracted/"`, means that even a raster in `"../data/01_extracted/dir1/dir2/"` will be clipped.

In [7]:

"""#Set clipping settings
clip_config = ClippingConfig(
    input_directory="../data/01_extracted/Datasets_2025Fall",  # where are your unclipped tiffs located?
    output_dir="../data/02_clipped/clipped_datasets_2025Fall", # where do you want the clipped tiffs to be saved?
    aoi_file="../data/aoi/TogGhaTanKen50km.geojson", # aoi defined in the previous step
    raster_pattern="*.tif",
    all_touched=True, # True: include all pixels touched by the AOI. False: only include pixels whose center points are in the AOI
)

#Run clipping functions
processor = ClippingProcessor(clip_config)
result = processor.process() 
"""

'#Set clipping settings\nclip_config = ClippingConfig(\n    input_directory="../data/01_extracted/Datasets_2025Fall",  # where are your unclipped tiffs located?\n    output_dir="../data/02_clipped/clipped_datasets_2025Fall", # where do you want the clipped tiffs to be saved?\n    aoi_file="../data/aoi/TogGhaTanKen50km.geojson", # aoi defined in the previous step\n    raster_pattern="*.tif",\n    all_touched=True, # True: include all pixels touched by the AOI. False: only include pixels whose center points are in the AOI\n)\n\n#Run clipping functions\nprocessor = ClippingProcessor(clip_config)\nresult = processor.process() \n'

In [8]:
align_config = AlignmentConfig(
    reference_raster="../data/02_clipped/clipped_datasets_2025Fall/WSF3D/WSF3D_V02_BuildingHeight.tif",
    input_directory="../data/02_clipped/clipped_datasets_2025Fall",
    output_dir="../data/03_processed/aligned_datasets_2025fall",
    recursive=True
)
processor = AlignmentProcessor(align_config) 
result = processor.process()    


2025-09-18 22:42:05,290 - INFO - AlignmentProcessor initialized
2025-09-18 22:42:05,293 - INFO - Enhanced NoData Detection: True
2025-09-18 22:42:05,294 - INFO - NoData Method: auto
2025-09-18 22:42:05,295 - INFO - Starting AlignmentProcessor processing
2025-09-18 22:42:05,300 - INFO - Starting raster alignment processing
2025-09-18 22:42:05,303 - INFO - Reference raster loaded: ../data/02_clipped/clipped_datasets_2025Fall/WSF3D/WSF3D_V02_BuildingHeight.tif
2025-09-18 22:42:05,303 - INFO -   CRS: EPSG:4326
2025-09-18 22:42:05,304 - INFO -   Shape: 30573 x 59278
2025-09-18 22:42:05,305 - INFO -   Resolution: 0.00 x 0.00
2025-09-18 22:42:05,307 - INFO -   Bounds: BoundingBox(left=-3.736055522222216, bottom=-12.174055599999996, right=42.3690555888889, top=11.604944400000008)
2025-09-18 22:42:05,310 - INFO - Found 7 raster files to align


Output()

2025-09-18 22:42:05,316 - INFO - Aligning rasters completed: 7/7 items in 0.0s (1247.1 items/sec)
2025-09-18 22:42:05,316 - INFO - Aligned 7/7 rasters successfully
2025-09-18 22:42:05,316 - INFO - Successfully completed AlignmentProcessor processing
